In [4]:
from Random_Walk import Random_Walk
from Shortest_Path import Shortest_Path
from Weisfeiler_Lehman import Weisfeiler_Lehman
#from WL_Version1 import Weisfeiler_Lehman

from WL_Transformer import WL_Transformer
from WL_edge import WL_edge
from WL_Shortest_Path import WL_Shortest_Path
from three_node_graphlet_kernel import Three_node_graphlet_kernel
import numpy as np 
import networkx as nx
from numpy.linalg import inv,multi_dot
from scipy.linalg import expm
from matplotlib import pyplot as plt
from networkx.drawing.nx_pydot import graphviz_layout
import pydot
from utils import hierarchy_pos,get_nxgraph_from_adjanecy_matrix,count_commons
from utils import load_data , create_loaders 
import copy
from collections import Counter
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

In [5]:
from autorank import autorank, plot_stats, create_report, latex_table
import pandas as pd

In [6]:
from torch_geometric.nn import global_mean_pool
import numpy as np
import networkx as nx
import os
import torchnet as tnt
import torch.nn.functional as F
import torch.nn as nn
import torch
import wget
import zipfile

In [5]:
np.random.seed(42)
pd.set_option('display.max_columns', 7)
std = 0.3
means = [0.2, 0.3, 0.5, 0.8, 0.85, 0.9]
sample_size = 50
data = pd.DataFrame()
for i, mean in enumerate(means):
    data['pop_%i' % i] = np.random.normal(mean, std, sample_size).clip(0, 1)

NameError: name 'pd' is not defined

In [ ]:
data

In [12]:
"""
# %%
wget.download(
    ")

file_name = 'MUTAG.zip'
zip_File = zipfile.ZipFile(file_name, 'r')
# 
zip_File.extractall()  

"""
# %%

'\n# %%\nwget.download(\n    ")\n\nfile_name = \'MUTAG.zip\'\nzip_File = zipfile.ZipFile(file_name, \'r\')\n# \nzip_File.extractall()  \n\n'

Node labels:

  0  C\
  1  N\
  2  O\
  3  F\
  4  I\
  5  Cl\
  6  Br\
\
Edge labels:\
\
  0  aromatic\
  1  single\
  2  double\
  3  triple\

In [9]:
dataset = load_data(path='./MUTAG/', ds_name='MUTAG',
                    use_node_labels=True, use_edge_labels=True, max_node_label=7,max_edge_label=4)


#pytorch loaders
train_dataset, val_dataset = create_loaders(
    dataset, batch_size=1, split_id=150, offset=0)
print('Data are ready')

Data are ready


In [10]:

X=np.array(dataset,dtype=object)[:,0]
y=np.array(dataset,dtype=object)[:,1]
G_train, G_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)

In [11]:
dataset[0][0].edges(data=True)

EdgeDataView([(1, 2, {'edge_label': array([1., 0., 0., 0.])}), (1, 6, {'edge_label': array([1., 0., 0., 0.])}), (2, 3, {'edge_label': array([1., 0., 0., 0.])}), (3, 4, {'edge_label': array([1., 0., 0., 0.])}), (4, 5, {'edge_label': array([1., 0., 0., 0.])}), (4, 10, {'edge_label': array([1., 0., 0., 0.])}), (5, 6, {'edge_label': array([1., 0., 0., 0.])}), (5, 7, {'edge_label': array([1., 0., 0., 0.])}), (7, 8, {'edge_label': array([1., 0., 0., 0.])}), (8, 9, {'edge_label': array([1., 0., 0., 0.])}), (9, 10, {'edge_label': array([1., 0., 0., 0.])}), (9, 14, {'edge_label': array([1., 0., 0., 0.])}), (10, 11, {'edge_label': array([1., 0., 0., 0.])}), (11, 12, {'edge_label': array([1., 0., 0., 0.])}), (12, 13, {'edge_label': array([1., 0., 0., 0.])}), (13, 14, {'edge_label': array([1., 0., 0., 0.])}), (13, 15, {'edge_label': array([0., 1., 0., 0.])}), (15, 16, {'edge_label': array([0., 0., 1., 0.])}), (15, 17, {'edge_label': array([0., 1., 0., 0.])})])

In [12]:
#get the first three graphs from the dataset
Gn=[dataset[0][0],dataset[1][0],dataset[2][0]]

In [13]:
y_test

array([1, 1, -1, 1, 1, 1, 1, -1, -1, -1, 1, -1, 1, -1, 1, 1, -1, 1, 1],
      dtype=object)

In [14]:
kernel=Three_node_graphlet_kernel(normalise=True, n_samples=500)
k_train=kernel.fit(list(G_train))
k_test=kernel.fit_transform(list(G_test))

# Uses the SVM classifier to perform classification
clf = SVC(kernel='precomputed')
clf.fit(np.asarray(k_train), np.ravel(y_train).astype(int))
y_pred = clf.predict(np.asarray(k_test))

# Computes and prints the classification accuracy
acc = accuracy_score(np.array(y_test,dtype=int), np.array(y_pred,dtype=int))
print("Accuracy:", str(round(acc*100, 2)) + "%")

Accuracy: 94.74%


In [15]:
G_train[2].number_of_edges()

25

In [16]:
sample_lentghs=[5,20,50,40,100,200]
pd.set_option('display.max_columns', 7)

populations=pd.DataFrame()

for i,n in enumerate(sample_lentghs):
    kernel=Three_node_graphlet_kernel(normalise=True, n_samples=n)
    k_train=kernel.fit(list(G_train))
    k_test=kernel.fit_transform(list(G_test))
    populations[f'{n}']=np.ravel(k_test)
    # Uses the SVM classifier to perform classification
    clf = SVC(kernel='precomputed')
    clf.fit(np.asarray(k_train), np.ravel(y_train).astype(int))
    y_pred = clf.predict(np.asarray(k_test))

    # Computes and prints the classification accuracy
    acc = accuracy_score(np.array(y_test,dtype=int), np.array(y_pred,dtype=int))
    print("Accuracy:", str(round(acc*100, 2)) + "%")

NameError: name 'pd' is not defined

In [60]:
populations

,5,20,50,40,100,200
0,0.894427,0.836163,0.917546,0.931050,0.995959,0.945609
1,0.674200,0.981025,0.991639,0.998540,0.960797,0.955212
2,0.992278,0.975741,0.999634,0.992874,0.996309,0.997578
3,0.868243,0.988985,0.994721,0.997816,0.996255,0.991607
4,0.989949,0.931857,0.958098,0.991520,0.998388,0.999462
...,...,...,...,...,...,...
3206,0.904534,0.996605,0.998421,0.999317,0.999372,0.997677
3207,0.707107,0.969773,0.985246,0.991666,0.995800,0.997309
3208,0.832050,0.991010,0.997760,0.999317,0.969433,0.999880
3209,0.554700,0.988833,0.983271,0.974079,0.996397,0.999713


In [66]:
result = autorank(populations, alpha=0.05, verbose=False)
print(result)
#result_bayesian = autorank(populations, alpha=0.05, verbose=False, approach='bayesian')#
#print(result_bayesian)
#create_report(result)


RankResult(rankdf=
     meanrank    median       mad  ci_lower  ci_upper effect_size   magnitude
5    4.904080  0.894427  0.095522  0.857493  0.894427         0.0  negligible
20   3.610557  0.972879  0.022471  0.969206  0.976777   -0.762595      medium
50   3.415135  0.977742  0.019194  0.974164  0.981205   -0.815664       large
40   3.368265  0.977485  0.018169  0.974866   0.98016   -0.814798       large
100  2.915914  0.984790  0.013067  0.982713  0.986791   -0.894028       large
200  2.786048  0.986845   0.01194  0.984536  0.988827   -0.915747       large
pvalue=0.0
cd=0.1330525664631019
omnibus=friedman
posthoc=nemenyi
all_normal=False
pvals_shapiro=[0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
homoscedastic=False
pval_homogeneity=0.0
homogeneity_test=levene
alpha=0.05
alpha_normality=0.008333333333333333
num_samples=3211
posterior_matrix=
None
decision_matrix=
None
rope=None
rope_mode=None
effect_size=akinshin_gamma)


In [ ]:
plot_stats(result)
plt.show()

In [78]:
nx.get_node_attributes(G_train[0], 'attr_dict')

{2186: array([1., 0., 0., 0., 0., 0., 0.]),
 2187: array([0., 1., 0., 0., 0., 0., 0.]),
 2188: array([1., 0., 0., 0., 0., 0., 0.]),
 2189: array([1., 0., 0., 0., 0., 0., 0.]),
 2190: array([1., 0., 0., 0., 0., 0., 0.]),
 2191: array([1., 0., 0., 0., 0., 0., 0.]),
 2192: array([1., 0., 0., 0., 0., 0., 0.]),
 2193: array([1., 0., 0., 0., 0., 0., 0.]),
 2194: array([1., 0., 0., 0., 0., 0., 0.]),
 2195: array([0., 1., 0., 0., 0., 0., 0.]),
 2196: array([0., 1., 0., 0., 0., 0., 0.]),
 2197: array([0., 0., 1., 0., 0., 0., 0.]),
 2198: array([0., 0., 1., 0., 0., 0., 0.])}

In [95]:
g=nx.convert_node_labels_to_integers(G_train[0])

In [96]:
nx.get_node_attributes(g, 'attr_dict')

{0: array([1., 0., 0., 0., 0., 0., 0.]),
 1: array([0., 1., 0., 0., 0., 0., 0.]),
 2: array([1., 0., 0., 0., 0., 0., 0.]),
 3: array([1., 0., 0., 0., 0., 0., 0.]),
 4: array([1., 0., 0., 0., 0., 0., 0.]),
 5: array([1., 0., 0., 0., 0., 0., 0.]),
 6: array([1., 0., 0., 0., 0., 0., 0.]),
 7: array([1., 0., 0., 0., 0., 0., 0.]),
 8: array([1., 0., 0., 0., 0., 0., 0.]),
 9: array([0., 1., 0., 0., 0., 0., 0.]),
 10: array([0., 1., 0., 0., 0., 0., 0.]),
 11: array([0., 0., 1., 0., 0., 0., 0.]),
 12: array([0., 0., 1., 0., 0., 0., 0.])}

In [92]:
nx.get_node_attributes(G_train[0], 'old')

{}

In [13]:
#reverse to integer labels
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
from sklearn.preprocessing import OneHotEncoder
enc = OneHotEncoder(handle_unknown='ignore')
enc.fit([[i] for i in range(7)])
new_g=[]
node_label='attr_dict'
for G in G_train:                
    G=nx.convert_node_labels_to_integers(G)
    # get the set of original labels
    labels_ori = list(nx.get_node_attributes(G, node_label).values())
    for i,node in enumerate(G.nodes(data=True)):
            node[1][node_label]=enc.inverse_transform([labels_ori[i]])[0][0]
            #print((node[1][node_label]))
    #print(nx.get_node_attributes(G, 'attr_dict').values())
    new_g.append(G)    
            


In [24]:
from utils import get_a_floyd_S_graph
S=get_a_floyd_S_graph(new_g[0])
S2=get_a_floyd_S_graph(new_g[1])
dict1=nx.get_node_attributes(new_g[0], 'attr_dict')
dict2=nx.get_node_attributes(new_g[1], 'attr_dict')
adj=nx.to_numpy_matrix(S,weight='cost')
adj2=nx.to_numpy_matrix(S2,weight='cost')
print(nx.to_numpy_matrix(S,weight='cost'))
tuple1= np.array(adj) , dict1 
tuple2=np.array(adj2), dict2

[[0. 1. 2. 3. 4. 5. 4. 3. 3. 2. 6. 7. 7.]
 [1. 0. 1. 2. 3. 4. 3. 2. 2. 1. 5. 6. 6.]
 [2. 1. 0. 1. 2. 3. 2. 1. 2. 2. 4. 5. 5.]
 [3. 2. 1. 0. 1. 2. 3. 2. 3. 3. 3. 4. 4.]
 [4. 3. 2. 1. 0. 1. 2. 3. 4. 4. 2. 3. 3.]
 [5. 4. 3. 2. 1. 0. 1. 2. 3. 4. 1. 2. 2.]
 [4. 3. 2. 3. 2. 1. 0. 1. 2. 3. 2. 3. 3.]
 [3. 2. 1. 2. 3. 2. 1. 0. 1. 2. 3. 4. 4.]
 [3. 2. 2. 3. 4. 3. 2. 1. 0. 1. 4. 5. 5.]
 [2. 1. 2. 3. 4. 4. 3. 2. 1. 0. 5. 6. 6.]
 [6. 5. 4. 3. 2. 1. 2. 3. 4. 5. 0. 1. 1.]
 [7. 6. 5. 4. 3. 2. 3. 4. 5. 6. 1. 0. 2.]
 [7. 6. 5. 4. 3. 2. 3. 4. 5. 6. 1. 2. 0.]]


In [14]:
from Shortest_Path_Labelled import Shortest_Path_Labelled
kernel=Shortest_Path_Labelled(node_label='attr_dict')
kernel.fit(new_g[:6])
kernel.fit_transform(new_g[6:10])


array([[0.34849094, 0.37637122, 0.37853028, 0.24975194, 0.42526195,
        0.31218992],
       [0.49698264, 0.43734589, 0.45345849, 0.2992826 , 0.32943817,
        0.50900716],
       [0.87262128, 0.66807418, 0.37296689, 0.82881531, 0.56450191,
        0.91687694],
       [0.34481746, 0.21170245, 0.61350793, 0.30203467, 0.57142857,
        0.30203467]])

In [1]:
def pairwise_operation(x, y):
        """Calculate shortests paths on attributes.
        Parameters
        ----------
        x, y : tuple
            Tuples of shortest path matrices and their attribute
            dictionaries.
        Returns
        -------
        kernel : number
            The kernel value.
        """
        # Initialise
        Sx, phi_x = x[0],x[1]
        Sy, phi_y = y[0],y[1]
        kernel = 0
        dimx = Sx.shape[0]
        dimy = Sy.shape[0]
        #matrix1 i row
        for i in range(dimx):
            #matrix1 j columns
            for j in range(dimx):
                #skip the diagonal as it s 0
                if i == j:
                    continue
                #else for rows k in second matrix
                for k in range(dimy):
                    #for   columns  m  second  matrix 
                    for m in range(dimy):
                        #agin skip the main diagonal 
                        if k == m:
                            continue
                        #else 
                        #if there s a matching distance at that entry, and it s not infinity
                        if (Sx[i, j] == Sy[k, m] and
                                Sx[i, j] != float('Inf')):
                            #add the dot product  between the node labels to the final kernel
                            kernel = kernel + np.dot(phi_x[i], phi_y[k]) * \
                                np.dot(phi_x[j], phi_y[m])

        return kernel

In [2]:
pairwise_operation(tuple1, tuple2)

NameError: name 'tuple1' is not defined

In [162]:
enc.transform([[0]]).toarray()

array([[1., 0., 0., 0., 0., 0., 0.]])

In [131]:
nx.get_node_attributes(G_train[1], 'attr_dict')

{1263: array([1., 0., 0., 0., 0., 0., 0.]),
 1264: array([1., 0., 0., 0., 0., 0., 0.]),
 1265: array([1., 0., 0., 0., 0., 0., 0.]),
 1266: array([1., 0., 0., 0., 0., 0., 0.]),
 1267: array([1., 0., 0., 0., 0., 0., 0.]),
 1268: array([1., 0., 0., 0., 0., 0., 0.]),
 1269: array([0., 1., 0., 0., 0., 0., 0.]),
 1270: array([0., 0., 1., 0., 0., 0., 0.]),
 1271: array([0., 0., 1., 0., 0., 0., 0.]),
 1272: array([1., 0., 0., 0., 0., 0., 0.]),
 1273: array([0., 0., 1., 0., 0., 0., 0.]),
 1274: array([1., 0., 0., 0., 0., 0., 0.]),
 1275: array([0., 0., 1., 0., 0., 0., 0.]),
 1276: array([0., 1., 0., 0., 0., 0., 0.])}